In [8]:
import torch
import pandas as pd
import os

In [13]:
path = "/Users/emilygong/Desktop/RUW-Oil-Prediction/data"
os.chdir(path)

In [53]:
guardian = pd.read_csv("Guardian_articles.csv")
guardian = guardian[['webPublicationDate', 'webTitle']]
nyt = pd.read_csv("NYT_articles.csv")
nyt = nyt[['headline', 'pub_date']]
ha = guardian.iat[1999,1]
ha

'Flashmob orchestra in Trafalgar Square plays in solidarity with Ukraine'

In [47]:
def clean(string):
    colon = string.find(":")
    comma = string.find("',")
    return string[colon+3:comma-1]

In [48]:
clean(ha)

'RUSSIA REPORTED BOLSTERING LINE5; Said to Be Increasing Forces in Ukraine, Poland and Estonia Against Reich AXIS ANNOYANCE IS SEEN Moscow Is Said to Have Told Nazis Bombers in the South Will \'Spray Wheat Crops\'", \'kicker\': None, \'content_kicker\': None, \'print_headline\': "RUSSIA REPORTED BOLSTERING LINE5; Said to Be Increasing Forces in Ukraine, Poland and Estonia Against Reich AXIS ANNOYANCE IS SEEN Moscow Is Said to Have Told Nazis Bombers in the South Will \'Spray Wheat Crops\'", \'name\': None, \'seo\': None, \'sub\': Non'

In [63]:
from transformers import pipeline

classifier = pipeline("text-generation", model = "gpt2")
output = classifier(ha, max_new_tokens = 100)[0]['generated_text']
output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Flashmob orchestra in Trafalgar Square plays in solidarity with Ukraine on September 10, 2016 (AFP Photo/Dmitry Kolesnikov)\n\n(Photo: Reuters)\n\nUkraine\'s National Anti-Crime Commission (NAACP) on Sunday questioned the president\'s "tearsome" comments when he said Ukraine\'s armed forces should be forced to provide more troops to troops in eastern Ukraine because of increased armed conflict in the country.\n\nIn an interview with an audience of over 1,000 people, Naftogaz deputy president Oleksandr'

In [64]:
sentiment = pipeline("sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")
original = sentiment(ha)
new = sentiment(output)
print(original)
print(new)

[{'label': 'POSITIVE', 'score': 0.9960684776306152}]
[{'label': 'NEGATIVE', 'score': 0.9613338708877563}]
